Load libraries

In [ ]:
import gc
import time
import numpy as np
import pandas as pd
import time
from typing import List, Dict, Set
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 修改当前文件夹位置 假定notebook文件就在项目文件夹根目录
import os
def get_root_dir():
    if os.path.exists('/content/drive/MyDrive/'):
        return '/content/drive/MyDrive/Colab/4-AMEX/AMEX Project/notebooks' #在Colab里
    else:
        return './' #在本地

#调用系统命令，相当于cd，但是直接!cd是不行的
os.chdir(get_root_dir())

## Concatenate Train & Test data

In [ ]:
train_df = pd.read_parquet('../data/4-PreCompressed/FilledWithRandomForest/train_RandomForest(PreCompressed).parquet', engine = 'pyarrow')
print(train_df.shape)
train_df.head(2)

(5531451, 173)


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,-1,-1,0,0,0.0,0,0.000610,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,-1,-1,0,0,0.0,0,0.005492,0


In [ ]:
test_df = pd.read_parquet('../data/4-PreCompressed/FilledWithRandomForest/test_RandomForest(PreCompressed).parquet', engine = 'pyarrow')
print(test_df.shape)
test_df.head(2)

(11363762, 173)


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-02-19,0.631315,0,0.010728,0.814497,0.0,0.168651,0.0,0.002347,...,-1,-1,-1,-1,-1,0,0.0,-1,0.008281,-1
1,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-03-25,0.587042,0,0.011026,0.810848,0.0,0.241389,0.0,0.009132,...,-1,-1,-1,-1,0,0,0.0,0,0.003753,0


In [ ]:
5531451 + 11363762

16895213

In [ ]:
df = pd.concat([train_df, test_df], axis=1)
print(df.shape)
del train_df
del test_df
gc.collect()

KeyboardInterrupt: ignored

In [ ]:
df.to_parquet('../data/4-PreCompressed/FilledWithRandomForest/full_filled.parquet', index=False)

## Load data

In [ ]:
df = pd.read_parquet('../data/4-PreCompressed/FilledWithRandomForest/full_filled.parquet', engine = 'pyarrow')
print(df.shape)
df.head(2)

(16895213, 173)


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,-1,-1,0,0,0.0,0,0.000610,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,-1,-1,0,0,0.0,0,0.005492,0


##Part I: Create interaction variables

In [ ]:
def create_interaction(df: pd.DataFrame, cat_cols: List[str], cont_cols: List[str]) -> pd.DataFrame:
  # dict for holding created variables
  features = {}
  card_all = df['customer_ID'].values.tolist()
  for card in card_all:
      features[card] = {}
  print("Starting ...")
  # create interaction variables
  num = 0
  t = time.time()
  for i, row in df.iterrows():
    card = row['customer_ID']
    for cat_col in cat_cols:
        for cont_col in cont_cols:
            col_name = '&'.join([cat_col, str(row[cat_col]), cont_col])
            features[card][col_name] = features[card].get(col_name, 0) + row[cont_col]
    num += 1
    if num % 100000 == 0:
      dur = round((time.time() - t) / 60, 2)
      print(f'{num} records processed in {dur} min.')

  # convert dict to df
  df = pd.DataFrame(features).T.reset_index()
  df.fillna(0, inplace=True)
  cols = df.columns.tolist()
  df.columns = ['customer_ID'] + cols[1:]
  return df

### Test with fake data

In [ ]:
# test create_interaction
t1 = {'customer_ID':[1, 2, 1, 3], 
      'A':[1, 2, 1, 2], 
      'B':[2, 1, 2, 2], 
      'C':[4, 5, 1, 5], 
      'D':[7, 5, 4, 8],}

t1 = pd.DataFrame(t1)
t1

In [ ]:
t1_new = create_interaction(t1, ['A', 'B'], ['C', 'D'])
# t1_new.to_parquet('/content/drive/My Drive/AMEX Project/data/temp.parquet', index=False)
t1_new


In [ ]:
cat_cols = ['B_30', 'B_38', 'D_64', 'D_68', 'D_114', 'D_117', 'D_120']
cont_cols = set(train_df.columns.tolist()) - set(cat_cols) - set(['customer_ID', 'S_2'])
cont_cols = list(cont_cols)
print(len(cont_cols))

In [ ]:
cont_cols_l = []
for i in range(0, 160, 20):
  # print(i)
  if i < 140:
    cont_cols_l.append(cont_cols[i:i+20])
  else:
    cont_cols_l.append(cont_cols[i:])
print(len(cont_cols_l))

In [ ]:
with open('../data/cont_ids.json', 'w') as f:
  json.dump(cont_cols_l, f)

### Run on actual data

In [ ]:
cat_cols = ['B_30', 'B_38', 'D_64', 'D_68', 'D_114', 'D_117', 'D_120']

In [ ]:
with open('../data/cont_ids.json', 'r') as f:
  cont_cols_l = json.load(f)
print(len(cont_cols_l))

8


In [ ]:
cont_cols = cont_cols_l[0]
half = len(cont_cols) // 2
cont_cols1, cont_cols2 = cont_cols[:half], cont_cols[half:]
cols1, cols2 = ['customer_ID'] + cat_cols + cont_cols1, ['customer_ID'] + cat_cols + cont_cols2
df1, df2 = df[cols1], df[cols2]
print(df1.shape, df2.shape)
del df
gc.collect()

(16895213, 18) (16895213, 18)


175

In [ ]:
df1 = create_interaction(df1, cat_cols, cont_cols1)
df1.to_parquet('../data/6-interaction/full_1_1.parquet', index=False)
print(df1.shape)

Starting ...
100000 records processed in 1.07 min.
200000 records processed in 2.05 min.
300000 records processed in 3.03 min.
400000 records processed in 4.0 min.
500000 records processed in 4.97 min.
600000 records processed in 5.94 min.
700000 records processed in 6.91 min.
800000 records processed in 7.87 min.
900000 records processed in 8.84 min.
1000000 records processed in 9.81 min.
1100000 records processed in 10.77 min.
1200000 records processed in 11.73 min.
1300000 records processed in 12.7 min.
1400000 records processed in 13.66 min.
1500000 records processed in 14.63 min.
1600000 records processed in 15.6 min.
1700000 records processed in 16.56 min.
1800000 records processed in 17.52 min.
1900000 records processed in 18.49 min.
2000000 records processed in 19.45 min.
2100000 records processed in 20.41 min.
2200000 records processed in 21.37 min.
2300000 records processed in 22.34 min.
2400000 records processed in 23.31 min.
2500000 records processed in 24.27 min.
2600000 r

In [ ]:
del df1
gc.collect()

66

In [ ]:
df2 = create_interaction(df2, cat_cols, cont_cols2)
df2.to_parquet('../data/6-interaction/full_1_2.parquet', index=False)
print(df2.shape)

Starting ...
100000 records processed in 1.07 min.
200000 records processed in 2.04 min.
300000 records processed in 3.0 min.
400000 records processed in 3.96 min.
500000 records processed in 4.93 min.
600000 records processed in 5.91 min.
700000 records processed in 6.86 min.
800000 records processed in 7.82 min.
900000 records processed in 8.79 min.
1000000 records processed in 9.75 min.
1100000 records processed in 10.79 min.
1200000 records processed in 11.79 min.
1300000 records processed in 12.79 min.
1400000 records processed in 13.78 min.
1500000 records processed in 14.78 min.
1600000 records processed in 15.77 min.
1700000 records processed in 16.77 min.
1800000 records processed in 17.76 min.
1900000 records processed in 18.75 min.
2000000 records processed in 19.76 min.
2100000 records processed in 20.78 min.
2200000 records processed in 21.78 min.
2300000 records processed in 22.79 min.
2400000 records processed in 23.78 min.
2500000 records processed in 24.8 min.
2600000 

## Part II: Compress continuous features and join with interaction variables

In [ ]:
def compress_features(df: pd.DataFrame, cat_cols: List[str], cont_cols: List[str]) -> pd.DataFrame:
  df = df.copy()
  cid = pd.Categorical(df.pop('customer_ID'), ordered=True) 
  last = (cid != np.roll(cid, -1)) # mask for last statement of every customer

  # compress continuous features: mean, min, max, last
  cont_df = df[['customer_ID'] + cont_cols]
  cont_avg = (cont_df
              .groupby('customer_ID')
              .mean()
              .rename(columns={f: f"{f}_avg" for f in cont_cols})
             )
  print('Avg has been computed')
  gc.collect()
  
  cont_max = (cont_df
            .groupby('customer_ID')
            .max()
            .rename(columns={f: f"{f}_max" for f in cont_cols})
            )
  print('Max has been computed')
  gc.collect()
  
  cont_min = (cont_df
            .groupby('customer_ID')
            .min()
            .rename(columns={f: f"{f}_min" for f in cont_cols})
            )
  print('Min has been computed')
  gc.collect()
  
  cont_last = (cont_df.loc[last, :]
              .rename(columns={f: f"{f}_last" for f in cont_cols})
              .set_index(np.asarray(cid[last]))
            )
  df = None # we no longer need the original data
  print('Last has been computed for ', i)


In [ ]:
['customer_ID'] + ['a', 'b', 'c']